<font size=6>Développer une preuve de concept</font>  
<font size=5>Classification d'image avec le papier Learning What and Where to Transfer  
Annexe : Extraction des feature maps les plus discriminantes</font>



---

**Vérification de l'environnement**

In [ ]:
import sys
IN_COLAB = "google.colab" in sys.modules
# PATH_DRIVE : to change according to your Google Drive folders
PATH_DRIVE = "/content/drive/My Drive/MachineLearning/ML07"
# IMAGES_DRIVE : to get access to previously loaded images
IMAGES_DRIVE = "/content/drive/My Drive/MachineLearning/ML06/Images"

In [ ]:
if IN_COLAB:
    print("Le notebook est exécuté sur Google Colab")
else:
    print("Le notebook est exécuté en local")

Le notebook est exécuté sur Google Colab


In [ ]:
if IN_COLAB:
    from google.colab import drive, files
    drive.mount("/content/drive")

Mounted at /content/drive


---
## <font color=blue>Notebook set-up</font>

**Importation des librairies**

In [ ]:
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import re
import datetime
import random as python_random

In [ ]:
import torch

In [ ]:
if IN_COLAB:
    sys.path.append(PATH_DRIVE)
    os.chdir(PATH_DRIVE)
    import sf_graphiques as sfg
    import sf_efficientnet_b4 as sf_efb4
    import sf_resnet50 as sf_resnet
    import sf_densenet201 as sf_densenet
    import sf_inception_v3 as sf_inception
    import sf_pytorch_loaders as sf_load
    import sf_l2t_extractfeatures as sfef
else:
    import modules_perso.sf_graphiques as sfg
    import modules_perso.sf_efficientnet_b4 as sf_efb4
    import modules_perso.sf_resnet50 as sf_resnet
    import modules_perso.sf_densenet201 as sf_densenet
    import modules_perso.sf_inception_v3 as sf_inception
    import modules_perso.sf_pytorch_loaders as sf_load
    import modules_perso.sf_l2t_extractfeatures as sfef

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


**Notebook set-up**

In [ ]:
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = fig_id + "." + fig_extension
    if IN_COLAB:
        path = PATH_DRIVE + "/" + path
    #print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, dpi=resolution)

In [ ]:
RANDOM_SEED = 42
BATCH_SIZE = 32

In [ ]:
def reset_random_seeds():
    np.random.seed(RANDOM_SEED)
    python_random.seed(RANDOM_SEED)
    torch.manual_seed(40)

In [ ]:
reset_random_seeds()

In [ ]:
if torch.cuda.device_count() >= 1:
    print(torch.cuda.get_device_name(0))
else:
    print("No Cuda Device")
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

Tesla P100-PCIE-16GB


---
## 1. Best Features d'EfficientNetB4

In [ ]:
model = sf_efb4.efficientnetb4(pretrained=True, max_num_features=512, folder_saves="models")

In [ ]:
best_features = sfef.BestFeatures(model, IMAGES_DRIVE, batch_size=32,
                                  with_data_augmentation=False, mini_data=0.2)

[2020-11-08 23:51:21,744] [main] /usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-ed8e3d68-08a9-4465-a345-36b5fdc1e68c.json


In [ ]:
best_features.calcul_best_features()

[2020-11-08 23:51:26,691] [main] Calculation is done


In [ ]:
best_features.model.extraction_num_features

[24, 32, 56, 160, 1792]

In [ ]:
best_features.model.extraction_best_features

[None,
 None,
 None,
 None,
 [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  15

In [ ]:
best_features.model.extraction_dimension

[112, 56, 28, 14, 7]

---
## Best Features d'EfficientNetB7

In [ ]:
efnb7_url = "models/efficientnet-b7-dcc49843.pth"
model = sf_efb4.efficientnetb7(pretrained=True, max_num_features=512,
                               folder_saves="models", url_pretrained=efnb7_url)
best_features = sfef.BestFeatures(model, IMAGES_DRIVE, batch_size=32,
                                  with_data_augmentation=False, mini_data=0.1)
best_features.calcul_best_features()

[2020-11-16 16:21:34,893] [main] /usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-357ac587-9934-4ba1-9560-072a2aa1b2bb.json


/content/drive/My Drive/MachineLearning/ML07/sf_features_extraction.py:196: RuntimeWarning: invalid value encountered in double_scalars
  / self.count[label_idx]


[2020-11-16 21:01:22,411] [main] Calculation is done


In [ ]:
best_features.model.extraction_num_features

[32, 48, 80, 224, 2560]

In [ ]:
best_features.model.extraction_best_features

[None,
 None,
 None,
 None,
 array([1601, 1600, 1918, 1917, 1916, 1915, 1914, 1913, 1912, 1911, 1910,
        1909, 1908, 1907, 1906, 1905, 1904, 1903, 1902, 1901, 1900, 1899,
        1898, 1897, 1896, 1895, 1894, 1893, 1892, 1891, 1890, 1889, 1888,
        1887, 1886, 1885, 1884, 1883, 1882, 1881, 1880, 1879, 1878, 1877,
        1876, 1875, 1874, 1873, 1872, 1871, 1870, 1869, 1868, 1867, 1866,
        1865, 1864, 1863, 1862, 1861, 1860, 1859, 1858, 1857, 1856, 1855,
        1854, 1853, 1852, 1851, 1850, 1849, 1848, 1847, 1846, 1845, 1844,
        1843, 1842, 1841, 1840, 1839, 1838, 1837, 1836, 1835, 1834, 1833,
        1832, 1831, 1830, 1829, 1828, 1827, 1826, 1825, 1824, 1823, 1822,
        1821, 1820, 1819, 1818, 1817, 1816, 1815, 1814, 1813, 1812, 1811,
        1810, 1809, 1808, 1807, 1806, 1805, 1804, 1803, 1802, 1801, 1800,
        1799, 1798, 1797, 1796, 1795, 1794, 1793, 1792, 1791, 1790, 1789,
        1788, 1787, 1786, 1785, 1784, 1783, 1782, 1781, 1780, 1779, 1778,
        17

In [ ]:
best_features.model.extraction_dimension

[112, 56, 28, 14, 7]